![Intel Logo](images/logo.png)

# MNIST Classification using OpenVINO 

### Contents :

   1. Preparing the MNIST Dataset
   
   2. Creating an OpenVINO application using the Inference Engine
   
   3. Visualize the results after inferring with actual data
   

## Preparing the MNIST Dataset
- Preparing and checking the MNIST Dataset

![Result](RES/L02/03.app_label.png)

In [ ]:
import os
USER = os.environ["USER"]
OVTRAINING = "/home/{}/openvino_training".format(USER)


In [ ]:
#Show the testset
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(image_train, label_train), (image_test, label_test) = mnist.load_data()
image_train, image_test = image_train / 255.0, image_test / 255.0

#show the 10 samples
plt.figure(figsize = (20,20))

for id in range(10):
    sp = plt.subplot(5,5,id+1)
    plt.imshow(image_test[id])
    plt.title(f'Id{[id]}, Label: {label_test[id]}')
    plt.axis('off')



## Refer to the figure and finish the code

- Refer to the following figure and fill in the blanks to produce a similar result

![Result](RES/L02/03.app_result.png)


In [ ]:
# import inferencce engine
???

# Ths network is trained with 96*96*3

#change (28,28,1) to (96*96*3)
def resize(img):
    ???
    return ???

# change 3D to 4D (including batch) and change order from NHWC to NCHW
def nchw(img):
    ???
    return ???


# prepare image from MNIST  and pre-process it
???
# resize image
???
# modify to imput image
???

# initialize Inference Engine (IE)
???

# read model and prepare network
???

# create executable network(Inference)
???

# What is the predicted value? (argmax)
print(output???)
print(???)


# Instructor reference 

In [ ]:
# Do Not Share
import os
from openvino.inference_engine import IECore

USER = os.environ["USER"]
input_size = 96

#change (28,28,1) to (96*96*3)
def resize(img):
    # resize the image first
    img = cv2.resize(img, (input_size, input_size))
    # expand 1-channel input image to 3-channels for input shape of MobileNetV2
    img = np.stack((img, )*3, axis=-1)
    # Make sure it is a float32
    return img
#    return img.astype(np.float32

# change 3D to 4D (including batch) and change order from NHWC to NCHW
def nchw(img):
    # Add Batch size(N) axis
    #img = img[np.newaxis, :, :, :]  # HWC to NHWC
    img = np.reshape(img, (1,96,96,3))
    #print(img.shape)
    reorder_img = img.transpose((0, 3, 1, 2))  # NHWC to NCHW
    #print(reorder_img.shape)
    return reorder_img

# initialize Inference Engine (IE)
ie = IECore()

# read model and prepare network


net = ie.read_network(model="/home/{}/openvino_training/mnist_mobilenetv2.xml".format(USER),
                      weights="/home/{}/openvino_training/mnist_mobilenetv2.bin".format(USER))

########################################################################
# create executable network
exec_net = ie.load_network(network=net,
                           device_name="CPU")

# prepare image from MNIST  and pre-process it
index = 7
image = nchw(resize(image_test[index]))

#print([s for s in dir(net.input_info['x']) if not s.startsWith('__')])
#print(type(net.input_info))
#print(iter(net.input_info))
#print(next(iter(net.input_info)))
#print(net.outputs)
#print(iter(net.outputs))
#print(next(iter(net.outputs)))
input_name = next(iter(net.input_info))
out_blob = next(iter(net.outputs))

plt.imshow(image_test[7])
infer_result = exec_net.infer({input_name: image})
#print(" * infer raw:",(infer_result))
#print(" * infer outpt",(infer_result[out_blob]))
#print label value
print(" * Label: ",label_test[index])
#print output
print(" * Result:", np.argmax(infer_result[out_blob]))
